In [ ]:
%matplotlib inline
import tensorflow as tf
import sys  
sys.path.insert(0, './')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam 
from deepexplain.tensorflow import DeepExplain
from sklearn.model_selection import KFold
#Local Libraries
from data_manager import DatasetManager
from attention_machine1 import AttentionNetwork
from feed_forward import FeedForward
import lime
import lime.lime_tabular
from lime import submodular_pick
from explain3 import call_L2X
from scipy.stats import weightedtau

In [ ]:
dm=DatasetManager(20000,10,version="additive_non_linear_with_product2",factor=1)

In [ ]:
Y,data=dm.generate_dataset(display=True)

In [ ]:
x=data[0:data.shape[0]*80//100]
x_test=data[data.shape[0]*80//100:]

In [ ]:
y=Y[0:data.shape[0]*80//100,:]
y_test=Y[data.shape[0]*80//100:,:]

In [ ]:
x_test.shape

In [ ]:
kf = KFold(n_splits=10)
kf.get_n_splits(x)

In [ ]:
def predict_fn(x):
    if len(x.shape)>1:
        pred=model.predict(x)
    else:
        pred=model.predict(np.array([x]))
    return np.array([1-pred[:,0],pred[:,0]]).T

In [ ]:
def get_top_k(array_deriv,k):
    topk=np.argsort(np.mean(array_deriv,axis=0))[::-1][:k]
    return topk

In [ ]:
def print_results(array,k,name,true=np.array([0,0,0,0,1,1,1,2,2,2])):
    print(name+":")
    mean=np.mean(array)
    median=np.median(array)
    std=np.std(array)
    print("Mean:"+str(mean))
    print("Median:"+str(median))
    print("Standard Dev:"+str(std))
    print("Standard Error:"+str(std/np.sqrt(len(array))))
    print("\n")
    return name,mean,median,std,std/np.sqrt(len(array))

In [ ]:
def compute_kendall(array,true):
    rank=np.argsort(array)[::-1]
    j=0
    print(rank)
    for i in rank:
        if j<=2:
            array[i]=2
        elif j>2 and j<6:
            array[i]=1
        else:
            array[i]=0
        j+=1
    result=weightedtau(array,true,weigher=lambda x: 1 if x<=6 else 0)[0]
    return result

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def sigmoid_d(x):
    return sigmoid(x)*(1-sigmoid(x))

In [ ]:
i=0
ranks_grad=[]
ranks_kax=[]
ranks_ka=[]
ranks_a_ka=[]
ranks_a_ka_tot=[]
ranks_ax=[]
ranks_a=[]
ranks_dl=[]
ranks_sa=[]
ranks_gi=[]
ranks_da=[]
ranks_LIME=[]
epoch=100
true=np.array([0,0,0,0,1,1,1,2,2,2])[::-1]
top_num=4
model_layers=(9,5,)
indexes=[]
for train_index, val_index in kf.split(x):
    print("Model:"+str(i))
    indexes.append([train_index,val_index])
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]
    tf.reset_default_graph()
    #Model
    gamma=0.0000001
    beta=0.00001
    inputs=tf.keras.Input(shape=(x_train.shape[1],))
    network=AttentionNetwork(num_feature=x_train.shape[1],list_reduc=model_layers,l2_strength=beta,dropout_act=False,classification=True)
    p,logit,AX,A,dA_dx,dp_dx,dH_dx=network(inputs)
    model=tf.keras.Model(inputs,p)
    #losses
    def loss_with_custom_regularization(y_true, y_pred):
        cross_entropy=tf.keras.losses.MSE(y_true, y_pred)
        a_regularization=tf.math.scalar_mul(gamma,tf.reduce_sum(tf.abs(A-1)))
        loss_loc=cross_entropy+a_regularization
        return loss_loc

    def loss_classication_with_custom_regularization(y_true, y_pred):
        cross_entropy=tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
        a_regularization=tf.math.scalar_mul(gamma,tf.reduce_sum(tf.abs(A-1)))
        loss_loc=cross_entropy+a_regularization
        return loss_loc

    def loss_classic(y_true, y_pred):
        loss_loc=tf.keras.losses.MSE(y_true, y_pred)
        return loss_loc

    def loss_classic_classification(y_true, y_pred):
        loss_loc=tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
        return loss_loc
    filepath="./attention_machine_non_linear_with_product2"+str(i)+".hdf5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',verbose=0, save_best_only=True, mode='min',save_weights_only=True)
    csv=tf.keras.callbacks.CSVLogger('./attention_machine_non_linear_with_product2'+str(i)+'.log')
    i+=1
    callbacks_list = [checkpoint,csv]
    adam = tf.keras.optimizers.Adam(lr = 0.01)
    model.compile(loss=loss_classication_with_custom_regularization,optimizer=adam,metrics=[loss_classic_classification])
    
    #Train Model
    print("Training...")
    model.fit(x_train, y_train, validation_data=(x_val, y_val),callbacks=callbacks_list, epochs=epoch, batch_size=512,verbose=True)
    
    #get the best model back
    model.load_weights(filepath)
    print("Training done!")
    model.evaluate(x_test, y_test)
    
    #output the desired values
    model2=tf.keras.Model(inputs,A)
    model3=tf.keras.Model(inputs,AX)
    model5=tf.keras.Model(inputs,dp_dx)
    a=model2.predict(x_test)
    h=model3.predict(x_test)
    derivative=model5.predict(x_test)
    w=network.logit.get_weights()
    grad=tf.keras.Model(inputs,dA_dx)
    grad_val=np.array(grad.predict(x_test))
    #Get dA_dX jacobian
    derivA=[]
    derivA_tot=[]
    derivA_alone=[]
    
    for j in range(x_test.shape[1]):
        sum_tot=w[0][:,0][j]*a[:,j]
        sum_tot_tot=w[0][:,0][j]*a[:,j]
        sum_da=0
        for k in range(x_test.shape[1]):
            topk=get_top_k(np.abs(w[0][:,0][k]*np.matmul(np.expand_dims(x_test[:,k],axis=-1).T,grad_val[k,:,:])),top_num)
            if j in topk:
                sum_tot+=w[0][:,0][k]*grad_val[k,:,j]*x_test[:,k]
            sum_tot_tot+=w[0][:,0][k]*grad_val[k,:,j]*x_test[:,k]
            sum_da+=w[0][:,0][k]*grad_val[k,:,j]*x_test[:,k]
        derivA.append(np.mean(np.abs(sum_tot*sigmoid_d(np.sum(w[0][:,0]*h,axis=1)))))
        derivA_tot.append(np.mean(np.abs(sum_tot_tot*sigmoid_d(np.sum(w[0][:,0]*h,axis=1)))))
        derivA_alone.append(np.mean(np.abs(sum_da)))
    #Get the ranking
    kax_loc=np.mean(np.abs(w[0][:,0]*h),axis=0)
    ka_loc=np.mean(np.abs(w[0][:,0]*a),axis=0)
    ax_loc=np.mean(np.abs(h),axis=0)
    a_loc=np.mean(np.abs(a),axis=0)
    dp_dx_loc=np.mean(np.abs(derivative),axis=0)
    a_da_loc=derivA
    a_da_tot_loc=derivA_tot
    ranks_grad.append(compute_kendall(dp_dx_loc,true))
    print("Grad:",ranks_grad[i-1])
    ranks_kax.append(compute_kendall(kax_loc,true))
    print("KAX:",ranks_kax[i-1])
    ranks_ka.append(compute_kendall(ka_loc,true))
    print("KA:",ranks_ka[i-1])
    ranks_a_ka.append(compute_kendall(a_da_loc,true))
    print("K(Ai+DAi/DXi*Xi):",ranks_a_ka[i-1])
    ranks_a_ka_tot.append(compute_kendall(a_da_tot_loc,true))
    print("K(Ai+sum(DAj/DXi*Xj)) :",ranks_a_ka_tot[i-1])
    ranks_da.append(compute_kendall(derivA_alone,true))
    print("K(sum(DAj/DXi*Xj)) :",ranks_da[i-1])
    ranks_ax.append(compute_kendall(ax_loc,true))
    print("AX:",ranks_ax[i-1])
    ranks_a.append(compute_kendall(a_loc,true))
    print("A:",ranks_a[i-1])
    
    #LIME
    explainer = lime.lime_tabular.LimeTabularExplainer(x_train , verbose=False,class_names=[str(x) for x in range(1)],discretize_continuous=True)
    pick=submodular_pick.SubmodularPick(explainer, x_test, predict_fn, num_features=10, num_exps_desired=1)
    lime_loc=[]
    for l in list(pick.explanations[0].local_exp.values())[0]:
        lime_loc.append(l[0])
    
    
    with DeepExplain(session=tf.keras.backend.get_session()) as de:
        # Need to reconstruct the graph in DeepExplain context, using the same weights.
        input_tensors = model.inputs[0]
        fModel = tf.keras.Model(model.inputs,model.output)
        target_tensor = fModel(input_tensors)
        dl = de.explain('deeplift', target_tensor, input_tensors,x_test)
        sa=de.explain('saliency', target_tensor, input_tensors,x_test)
        gi=de.explain('grad*input', target_tensor, input_tensors,x_test)
    dl_loc=np.mean(np.abs(dl),axis=0)
    sa_loc=np.mean(np.abs(sa),axis=0)
    gi_loc=np.mean(np.abs(gi),axis=0)
    ranks_dl.append(compute_kendall(dl_loc,true))
    print("Deep Lift:",ranks_dl[i-1])
    ranks_sa.append(compute_kendall(sa_loc,true))
    print("Saliency:",ranks_sa[i-1])
    ranks_gi.append(compute_kendall(gi_loc,true))
    print("Grad*Input:",ranks_gi[i-1])
    ranks_LIME.append(compute_kendall(lime_loc,true))
    print("LIME:",ranks_LIME[i-1])


In [ ]:
#Additive multi product
results_circular=[]
n=6
#results_circular.append(print_results(ranks_grad,n,"dp_dx"))
results_circular.append(print_results(ranks_kax,n,"KAX on AM"))
results_circular.append(print_results(ranks_ka,n,"KA on AM"))
results_circular.append(print_results(ranks_a_ka,n,"KiAi+Kj*dAj/dXi*Xj {j \ j in topk} on AM"))
results_circular.append(print_results(ranks_a_ka_tot,n,"Ki*Ai+Kj*dAj/dXi*Xj {j=0 to N features} on AM"))
results_circular.append(print_results(ranks_da,n,"Kj*dAj/dXi*Xj {j=0 to N features} on AM"))
results_circular.append(print_results(ranks_ax,n,"AX on AM"))
results_circular.append(print_results(ranks_a,n,"A on AM"))
results_circular.append(print_results(ranks_dl,n,"Deep Lift on AM"))
results_circular.append(print_results(ranks_sa,n,"Saliency on AM"))
results_circular.append(print_results(ranks_gi,n,"Grad*Input on AM"))
results_circular.append(print_results(ranks_LIME,n,"LIME on AM"))

In [ ]:
i=0
ranks_dlc=[]
ranks_sac=[]
ranks_gic=[]
ranks_dp_dxc=[]
ranks_L2X=[]
ranks_LIMEc=[]
model_layers=(9,5,10)
epoch=100
for train_index, val_index in indexes:
    print("Model:"+str(i))
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]
    tf.reset_default_graph()
    #Model
    inputs=tf.keras.Input(shape=(x_train.shape[1],))
    inputs=tf.keras.Input(shape=(x.shape[1],))
    network=FeedForward(num_feature=x.shape[1],list_reduc=model_layers,classification=True)
    p,logit,dp_dx=network(inputs)
    model=tf.keras.Model(inputs,p)
    #losses
    def loss_classic(y_true, y_pred):
        loss_loc=tf.keras.losses.MSE(y_true, y_pred)
        return loss_loc

    def loss_classic_classification(y_true, y_pred):
        loss_loc=tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
        return loss_loc
    filepath="./feed_forward_non_linear_with_product2"+str(i)+".hdf5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',verbose=0, save_best_only=True, mode='min',save_weights_only=True)
    csv=tf.keras.callbacks.CSVLogger('./feed_forward_non_linear_with_product2'+str(i)+'.log')
    i+=1
    callbacks_list = [checkpoint,csv]
    adam = tf.keras.optimizers.Adam(lr = 0.01)
    model.compile(loss=loss_classic_classification,optimizer=adam,metrics=[loss_classic_classification])
    
    #Train Model
    print("Training...")
    model.fit(x_train, y_train, validation_data=(x_val, y_val),callbacks=callbacks_list, epochs=epoch, batch_size=512,verbose=False)
    
    #get the best model back
    model.load_weights(filepath)
    print("Training done!")
    model.evaluate(x_test, y_test)
    
    #output the desired values
    model5=tf.keras.Model(inputs,dp_dx)
    derivative=model5.predict(x_test)
    
    #Get the ranking
    with DeepExplain(session=tf.keras.backend.get_session()) as de:
        # Need to reconstruct the graph in DeepExplain context, using the same weights.
        input_tensors = model.inputs[0]
        fModel = tf.keras.Model(model.inputs,model.output)
        target_tensor = fModel(input_tensors)
        dl = de.explain('deeplift', target_tensor, input_tensors,x_test)
        sa=de.explain('saliency', target_tensor, input_tensors,x_test)
        gi=de.explain('grad*input', target_tensor, input_tensors,x_test)
    dl_loc=np.mean(np.abs(dl),axis=0)
    sa_loc=np.mean(np.abs(sa),axis=0)
    gi_loc=np.mean(np.abs(gi),axis=0)
    dp_dx_loc=np.mean(np.abs(derivative),axis=0)
    
    #L2X
    print("L2X")
    _,L2X_loc=call_L2X([x_train,np.array([1-y_train[:,0],y_train[:,0]]).T,
              x_val,np.array([1-y_val[:,0],y_val[:,0]]).T,
              x_test,np.array([1-y_test[:,0],y_test[:,0]]).T,
              "additive_non_linear"])
    
    #LIME
    explainer = lime.lime_tabular.LimeTabularExplainer(x_train , verbose=False,class_names=[str(x) for x in range(1)],discretize_continuous=True)
    pick=submodular_pick.SubmodularPick(explainer, x_test, predict_fn, num_features=10, num_exps_desired=1)
    lime_loc=[]
    for l in list(pick.explanations[0].local_exp.values())[0]:
        lime_loc.append(l[0])
    ranks_dlc.append(compute_kendall(dl_loc,true))
    print("Deep Lift:",ranks_dlc[i-1])
    ranks_sac.append(compute_kendall(sa_loc,true))
    print("Saliency:",ranks_sac[i-1])
    ranks_gic.append(compute_kendall(gi_loc,true))
    print("Grad*Input:",ranks_gic[i-1])
    ranks_L2X.append(compute_kendall(L2X_loc,true))
    print("L2X:",ranks_L2X[i-1])
    ranks_LIMEc.append(compute_kendall(lime_loc,true))
    print("LIME:",ranks_LIMEc[i-1])

In [ ]:
results_circular.append(print_results(ranks_dlc,n,"Deep Lift on FF"))
results_circular.append(print_results(ranks_sac,n,"Saliency on FF"))
results_circular.append(print_results(ranks_gic,n,"Grad*Input on FF"))
results_circular.append(print_results(ranks_L2X,n,"L2X on FF"))
results_circular.append(print_results(ranks_LIMEc,n,"LIME on FF"))